In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

# set the style of the plots
# sns.set(style="whitegrid")



### Load Data 
 - Load the age-specific population data 
 - Make it read for processing  

In [10]:
# load the data set from xlsx file
full_data = pd.read_excel('./age_pop_world.xlsx', skiprows=16)

full_data.head()



,Index,Variant,region,Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+
0,1,Estimates,World,NaN,900,NaN,NaN,1.0,World,0,...,84810.2435,70169.8575,52277.0795,36002.3945,22005.951,10799.1055,4049.087,1010.136,160.347,14.593
1,2,Estimates,World,NaN,900,NaN,NaN,1.0,World,0,...,86649.7785,70795.911,53158.673,36425.648,22292.1835,10814.261,4021.9485,1008.231,153.134,14.9565
2,3,Estimates,World,NaN,900,NaN,NaN,1.0,World,0,...,88670.7395,71386.4255,54251.183,36941.9155,22538.043,10893.917,4009.361,1003.6405,151.8765,15.411
3,4,Estimates,World,NaN,900,NaN,NaN,1.0,World,0,...,90752.969,72028.5805,55578.4835,37587.1765,22775.071,11047.277,4010.387,1004.0725,150.944,15.4325
4,5,Estimates,World,NaN,900,NaN,NaN,1.0,World,0,...,92762.6905,73006.321,56840.661,38336.3855,23024.3465,11208.3675,4035.835,1009.044,150.4255,14.9595


### Process data 
- Filter data for Vietnam
- check select only those columns that are relvant
- make fewer age classes (?)

In [11]:
# filter out only data for vietnam
viet_data = full_data[full_data['region'] == 'Viet Nam']
viet_data = viet_data.loc[:, 'Year':]

viet_data['Total'] = viet_data.loc[:, '0-4':'100+'].replace('...', 0).astype(float).sum(axis=1)

viet_data.head()

,Year,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,...,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+,Total
9920,1950.0,3407.4965,2452.679,2375.6825,2520.2405,2182,2045.393,1777.428,1701.326,1465.019,...,691.691,484.1255,308.6025,160.251,51.948,21.5185,6.26,1.176,0.1715,25100.7380
9921,1951.0,3661.573,2505.5765,2347.7475,2512.7685,2220.5295,2060.343,1798.7395,1707.99,1479.4505,...,713.8205,501.6855,320.1775,170.1585,57.792,19.889,5.946,1.0165,0.15,25600.2685
9922,1952.0,3938.571,2577.4115,2335.7345,2475.083,2282.2715,2059.0965,1836.1365,1699.3925,1512.3725,...,736.5715,520.378,331.7525,179.4655,65.0515,18.2885,5.761,0.9625,0.135,26143.9175
9923,1953.0,4248.156,2681.1355,2340.944,2418.2305,2353.0115,2051.7885,1880.7455,1685.1895,1554.285,...,759.337,539.8915,343.7195,188.3585,72.725,17.4325,5.583,0.9075,0.1175,26757.9645
9924,1954.0,4483.1155,2892.3675,2361.454,2365.272,2407.304,2059.6805,1920.7445,1681.8165,1589.4805,...,781.491,559.7505,356.542,197.0185,79.758,17.988,5.2505,0.827,0.0955,27427.9085


- Melt into long form for plotting later 

In [15]:
viet_data_long = (
    viet_data
    .melt(id_vars=['Year', 'Total'], 
               value_vars=['0-4'  , '5-9'  , '10-14', '15-19', '20-24', 
                           '25-29', '30-34', '35-39', '40-44', '45-49', 
                           '50-54', '55-59', '60-64', '65-69', '70-74', 
                           '75-79', '80-84', '85-89', '90-94', '95-99', 
                           '100+'], 
               var_name='Age', 
               value_name='Population')

)

viet_data_long = (
    viet_data_long
    # .groupby('Age')
    .assign(
        prop_of_total = lambda x: x['Population'] / x['Total']
        )
    .reset_index()
    )

In [31]:
age_data_females = pd.read_excel('./vietnam_females_age.xlsx', skiprows=1)
age_data_females = age_data_females.loc[:, 'Year':]
age_data_females['Sex'] = 0



age_data_males = pd.read_excel('./vietnam_males_age.xlsx', skiprows=1)
age_data_males = age_data_males.loc[:, 'Year':]
age_data_males['Sex'] = 1


age_data = pd.concat([age_data_females, age_data_males], ignore_index=True)


age_data.columns = ['Year'] + list(range(101)) + ['Sex']

age_data_long = (
    age_data
    .melt(id_vars=['Year', 'Sex'], 
          value_vars=list(range(101)), 
          var_name='Age', 
          value_name='Population')
          )


age_data_long.groupby(['Age', 'Year']).agg({'Population': 'sum'}).reset_index()




Population
Age Year            
0   1950    905.4295
    1951    873.6500
    1952    917.7490
    1953    989.4810
    1954   1036.9485
...              ...
100 2019      5.3235
    2020      5.8575
    2021      6.1520
    2022      6.2815
    2023      6.7015

[7474 rows x 1 columns]